In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:

def extraer_datos(urlPart, cantidaddeano, anoInicio):
    cont = 0
    dict_anyo_smi = dict()
    datos = []

    while cont < cantidaddeano:
        url = f"{urlPart}{anoInicio}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        
        for s in soup.find_all("tr")[1:]:
            datos.append([x.text for x in s.find_all("td")])
        anoInicio -= 1
        cont += 1
    
    return datos


In [3]:
datos=extraer_datos('https://datosmacro.expansion.com/smi?anio=',20,2023)

In [4]:
def crear_DataFrame(datos_extraidos, columnas):
    df = pd.DataFrame(datos_extraidos, columns=columnas)
    return df


#Llamamos a la funciòn "crear_DataFrame" para crear un DataFrame con los datos extraidos de la web

columnas = ["Pais", "Año", "SMI Local", "Divisa", "Dolares($)", "SMI(€)", "Vacia", "Var SMI(%)"]

df_paises = crear_DataFrame(datos,columnas)

    
#Eliminaciòn de la columna "Vacia", ya que no contine datos.
df_paises = df_paises.drop("Vacia", axis = 1)

df_paises

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%)
0,España [+],2023,"1.260,0",€,"1.343,9 $","1.260,0 €","8,00%"
1,Alemania [+],2023,"1.987,0",€,"2.119,3 $","1.987,0 €","14,86%"
2,Reino Unido [+],2022,"1.621,1",£,"1.952,7 $","1.929,2 €","12,91%"
3,Francia [+],2023,"1.709,3",€,"1.823,1 $","1.709,3 €","3,87%"
4,Portugal [+],2023,"886,7",€,"945,7 $","886,7 €","7,80%"
...,...,...,...,...,...,...,...
3028,Uzbekistán [+],2004,"6.530,0",л,"6,7 $","5,4 €",
3029,Venezuela [+],2004,"321,0",Bs,,,
3030,Viet Nam [+],2004,"290.000,0",₫,"18,4 $",,
3031,Sudáfrica [+],2004,"1.310,0",R,"202,8 $","157,3 €","17,71%"


In [5]:
#Limpieza de la columa pais.
df_paises['Pais'] = [x.replace('[+]','').replace(' ','') for x in df_paises['Pais']]

#No hace falta convertir los valores del SMI Local ya que estàn en distintas divisas
#Para operaciones usaremos las columnas Dolares y SMI

df_paises

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%)
0,España,2023,"1.260,0",€,"1.343,9 $","1.260,0 €","8,00%"
1,Alemania,2023,"1.987,0",€,"2.119,3 $","1.987,0 €","14,86%"
2,ReinoUnido,2022,"1.621,1",£,"1.952,7 $","1.929,2 €","12,91%"
3,Francia,2023,"1.709,3",€,"1.823,1 $","1.709,3 €","3,87%"
4,Portugal,2023,"886,7",€,"945,7 $","886,7 €","7,80%"
...,...,...,...,...,...,...,...
3028,Uzbekistán,2004,"6.530,0",л,"6,7 $","5,4 €",
3029,Venezuela,2004,"321,0",Bs,,,
3030,VietNam,2004,"290.000,0",₫,"18,4 $",,
3031,Sudáfrica,2004,"1.310,0",R,"202,8 $","157,3 €","17,71%"


In [6]:
#Limpieza y conversion (a float) de la columna Dolares($)

df_paises ["Dolares($)"] = [x.replace('.','').replace(',','.').replace('\xa0$','') if type(x)!=float else x for x in df_paises ["Dolares($)"]]

#Detectamos que en esta columna aparece aparecen elemento con '€'. Procedemos con la limpieza correspondiente.      
df_paises ["Dolares($)"] = [x.replace('€','') if type(x)!=float  else x for x in df_paises ["Dolares($)"]] 

df_paises ["Dolares($)"] = [np.nan if (x==None or x=='') else x for x in df_paises ['Dolares($)']]

#Finalmente pasamos a float el resto de valores de la tabla
df_paises ["Dolares($)"] = [float(x) if type(x)!=float else x for x in df_paises ["Dolares($)"]] 

df_paises  

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%)
0,España,2023,"1.260,0",€,1343.9,"1.260,0 €","8,00%"
1,Alemania,2023,"1.987,0",€,2119.3,"1.987,0 €","14,86%"
2,ReinoUnido,2022,"1.621,1",£,1952.7,"1.929,2 €","12,91%"
3,Francia,2023,"1.709,3",€,1823.1,"1.709,3 €","3,87%"
4,Portugal,2023,"886,7",€,945.7,"886,7 €","7,80%"
...,...,...,...,...,...,...,...
3028,Uzbekistán,2004,"6.530,0",л,6.7,"5,4 €",
3029,Venezuela,2004,"321,0",Bs,NaN,,
3030,VietNam,2004,"290.000,0",₫,18.4,,
3031,Sudáfrica,2004,"1.310,0",R,202.8,"157,3 €","17,71%"


In [7]:
#Limpieza y conversion (a float) de la columna SMI(€)

#Pasamos a Nan los None y las cadenas vacias
df_paises ["SMI(€)"] = [np.nan if (x==None or x=='') else x for x in df_paises ["SMI(€)"]]
     
#Limpiamos los elementos de la columna, para poder luego convertirlos a Float
df_paises ["SMI(€)"] = [x.replace('\xa0','').replace('.','').replace(',','.').replace('€','') if type(x)!=float  else x for x in df_paises ["SMI(€)"]] 
    
#Finalmente pasamos a float el resto de valores de la tabla
df_paises ["SMI(€)"] = [float(x) if type(x)!=float else x for x in df_paises ["SMI(€)"]] 
#Pasamos a Nan los None y las cadenas vacias
df_paises ["SMI Local"] = [np.nan if (x==None or x=='') else x for x in df_paises ["SMI Local"]]
     
#Limpiamos los elementos de la columna, para poder luego convertirlos a Float
df_paises ["SMI Local"] = [x.replace('\xa0','').replace('.','').replace(',','.').replace('€','') if type(x)!=float  else x for x in df_paises ["SMI Local"]] 
    
#Finalmente pasamos a float el resto de valores de la tabla
df_paises ["SMI Local"] = [float(x) if type(x)!=float else x for x in df_paises ["SMI Local"]] 

#Pasamos los años a int
df_paises ["Año"] = [int(x) if type(x)!=int else x for x in df_paises ["Año"]] 
df_paises

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%)
0,España,2023,1260.0,€,1343.9,1260.0,"8,00%"
1,Alemania,2023,1987.0,€,2119.3,1987.0,"14,86%"
2,ReinoUnido,2022,1621.1,£,1952.7,1929.2,"12,91%"
3,Francia,2023,1709.3,€,1823.1,1709.3,"3,87%"
4,Portugal,2023,886.7,€,945.7,886.7,"7,80%"
...,...,...,...,...,...,...,...
3028,Uzbekistán,2004,6530.0,л,6.7,5.4,
3029,Venezuela,2004,321.0,Bs,NaN,NaN,
3030,VietNam,2004,290000.0,₫,18.4,NaN,
3031,Sudáfrica,2004,1310.0,R,202.8,157.3,"17,71%"


In [8]:
#Limpieza y conversion (a float) de la columna Var SMI(%)

#Pasamos a Nan los None y las cadenas vacias
df_paises ["Var SMI(%)"] = [np.nan if (x==None or x=='') else x for x in df_paises ["Var SMI(%)"]]

#Limpiamos los elementos de la columna, para poder luego convertirlos a Float
df_paises ["Var SMI(%)"] = [x.replace('%','').replace(',','.').replace('.','') if type(x)!=float else x for x in df_paises ["Var SMI(%)"]]

     
#Finalmente pasamos a float el resto de valores de la tabla
df_paises ["Var SMI(%)"] = [float(x)/100 if type(x)!=float else x for x in df_paises ["Var SMI(%)"]] 


df_paises

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%)
0,España,2023,1260.0,€,1343.9,1260.0,8.00
1,Alemania,2023,1987.0,€,2119.3,1987.0,14.86
2,ReinoUnido,2022,1621.1,£,1952.7,1929.2,12.91
3,Francia,2023,1709.3,€,1823.1,1709.3,3.87
4,Portugal,2023,886.7,€,945.7,886.7,7.80
...,...,...,...,...,...,...,...
3028,Uzbekistán,2004,6530.0,л,6.7,5.4,NaN
3029,Venezuela,2004,321.0,Bs,NaN,NaN,NaN
3030,VietNam,2004,290000.0,₫,18.4,NaN,NaN
3031,Sudáfrica,2004,1310.0,R,202.8,157.3,17.71


In [9]:

df_paises[df_paises["Pais"]=="Alemania"]

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%)
1,Alemania,2023,1987.0,€,2119.3,1987.0,14.86
172,Alemania,2022,1730.0,€,1853.4,1730.0,6.40
343,Alemania,2021,1602.0,€,1954.6,1602.0,1.07
514,Alemania,2020,1544.0,€,1763.6,1544.0,-1.09
685,Alemania,2019,1561.0,€,1747.5,1561.0,3.65
854,Alemania,2018,1506.0,€,1778.5,1506.0,0.00
1023,Alemania,2017,1506.0,€,1701.3,1506.0,4.29
1192,Alemania,2016,1444.0,€,1598.4,1444.0,0.00
1360,Alemania,2015,1444.0,€,1602.1,1444.0,NaN


In [10]:
df_paises.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pais        3033 non-null   object 
 1   Año         3033 non-null   int64  
 2   SMI Local   3024 non-null   float64
 3   Divisa      3033 non-null   object 
 4   Dolares($)  2959 non-null   float64
 5   SMI(€)      2058 non-null   float64
 6   Var SMI(%)  1959 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 166.0+ KB


In [11]:
paises_dic = {'Europa': ['Albania', 'Alemania', 'Andorra', 'Armenia', 'Azerbaiyán', 'Bielorrusia', 'BosniayHerzegovina', 'Bulgaria', 'Bélgica', 'Chequia', 'Chipre', 'Croacia', 'Eslovaquia', 'Eslovenia', 'España', 'Estonia', 'Francia', 'Georgia', 'Grecia', 'Hungría', 'Irlanda', 'Islandia', 'Israel', 'Kazajistán', 'Letonia', 'Lituania', 'Luxemburgo', 'MacedoniadelNorte', 'Malta', 'Moldavia', 'Montenegro', 'PaísesBajos', 'Polonia', 'Portugal', 'ReinoUnido', 'Rumanía', 'Rusia', 'SanMarino', 'Serbia', 'Suiza', 'Ucrania', 'Uzbekistán'],
               'Asia': ['Afganistán', 'ArabiaSaudita', 'Bangladés', 'Baréin', 'Bután', 'Camboya', 'Catar', 'China', 'CoreadelSur', 'EstadodePalestina', 'Filipinas', 'HongKong', 'India', 'Indonesia', 'Irak', 'Irán', 'Japón', 'Jordania', 'Kirguistán', 'Kuwait', 'Laos', 'Líbano', 'Malasia', 'Maldivas', 'Mongolia', 'Myanmar', 'Nepal', 'Omán', 'Pakistán', 'Palaos', 'Samoa', 'Siria', 'SriLanka', 'Surinam', 'Tailandia', 'Taiwan', 'Tayikistán', 'TimorOriental', 'Turkmenistán', 'Türkiye', 'VietNam'],
               'Norteamerica': ['AntiguayBarbuda', 'Bahamas', 'Barbados', 'Belice', 'Canadá', 'Cuba', 'Dominica', 'EstadosUnidos', 'Granada', 'Haití', 'Jamaica', 'México', 'RepúblicaDominicana', 'SanCristóbalyNieves', 'SanVicenteylasGranadinas', 'TrinidadyTobago'],
               'America_Central': ['CostaRica', 'ElSalvador', 'Guatemala', 'Honduras', 'Nicaragua', 'Panamá'],
               'Sudamerica': ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Perú', 'Uruguay', 'Venezuela'], 
               'Africa': ['Angola', 'Argelia', 'Benin', 'Botsuana', 'BurkinaFaso', 'Burundi', 'CaboVerde', 'Camerún', 'Chad', 'Comoras', 'CostadeMarfil', 'Egipto', 'Eswatini', 'Etiopía', 'Gabón', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bisáu', 'GuineaEcuatorial', 'Kenia', 'Lesoto', 'Liberia', 'Libia', 'Madagascar', 'Malaui', 'Malí', 'Marruecos', 'Mauricio', 'Mauritania', 'Mozambique', 'Nigeria', 'Níger', 'RepúblicaCentroafricana', 'RepúblicaDemocráticadelCongo', 'RepúblicadelCongo', 'Ruanda', 'Senegal', 'Seychelles', 'SierraLeona', 'Sudáfrica', 'Sudán', 'Tanzania', 'Togo', 'Túnez', 'Uganda', 'Zambia'],
               'Oceania': ['Australia', 'Fiyi', 'IslasMarshall', 'IslasSalomón', 'Kiribati', 'NuevaZelanda', 'PapúaNuevaGuinea', 'Vanuatu']
              }

In [12]:
Europa = paises_dic['Europa']
Asia = paises_dic['Asia']
Norteamerica = paises_dic['Norteamerica']
America_Central = paises_dic['America_Central']
Sudamerica = paises_dic['Sudamerica']
Africa = paises_dic['Africa']
Oceania = paises_dic['Oceania']

In [13]:
lista_paises = ['España', 'Alemania', 'ReinoUnido', 'Francia', 'Portugal', 'EstadosUnidos', 'Japón', 'China', 'Andorra', 'Afganistán', 'AntiguayBarbuda', 'Albania', 'Armenia', 'Angola', 'Argentina', 'Australia', 'Azerbaiyán', 'BosniayHerzegovina', 'Barbados', 'Bangladés', 'Bélgica', 'BurkinaFaso', 'Bulgaria', 'Baréin', 'Burundi', 'Benin', 'Bolivia', 'Brasil', 'Bahamas', 'Bután', 'Botsuana', 'Bielorrusia', 'Belice', 'Canadá', 'RepúblicaDemocráticadelCongo', 'RepúblicaCentroafricana', 'RepúblicadelCongo', 'Suiza', 'CostadeMarfil', 'Chile', 'Camerún', 'Colombia', 'CostaRica', 'Cuba', 'CaboVerde', 'Chipre', 'Chequia', 'Dominica', 'RepúblicaDominicana', 'Argelia', 'Ecuador', 'Estonia', 'Egipto', 'Etiopía', 'Fiyi', 'Gabón', 'Granada', 'Georgia', 'Ghana', 'Gambia', 'Guinea', 'GuineaEcuatorial', 'Grecia', 'Guatemala', 'Guinea-Bisáu', 'Guyana', 'HongKong', 'Honduras', 'Croacia', 'Haití', 'Hungría', 'Indonesia', 'Irlanda', 'Israel', 'India', 'Irak', 'Irán', 'Islandia', 'Jamaica', 'Jordania', 'Kenia', 'Kirguistán', 'Camboya', 'Kiribati', 'Comoras', 'SanCristóbalyNieves', 'CoreadelSur', 'Kuwait', 'Kazajistán', 'Laos', 'Líbano', 'SriLanka', 'Liberia', 'Lesoto', 'Lituania', 'Luxemburgo', 'Letonia', 'Libia', 'Marruecos', 'Moldavia', 'Montenegro', 'Madagascar', 'IslasMarshall', 'MacedoniadelNorte', 'Malí', 'Myanmar', 'Mongolia', 'Mauritania', 'Malta', 'Mauricio', 'Maldivas', 'Malaui', 'México', 'Malasia', 'Mozambique', 'Níger', 'Nigeria', 'Nicaragua', 'PaísesBajos', 'Nepal', 'NuevaZelanda', 'Omán', 'Panamá', 'Perú', 'PapúaNuevaGuinea', 'Filipinas', 'Pakistán', 'Polonia', 'EstadodePalestina', 'Palaos', 'Paraguay', 'Catar', 'Rumanía', 'Serbia', 'Rusia', 'Ruanda', 'ArabiaSaudita', 'IslasSalomón', 'Seychelles', 'Sudán', 'Eslovenia', 'Eslovaquia', 'SierraLeona', 'SanMarino', 'Senegal', 'Surinam', 'ElSalvador', 'Siria', 'Eswatini', 'Chad', 'Togo', 'Tailandia', 'Tayikistán', 'TimorOriental', 'Turkmenistán', 'Túnez', 'Türkiye', 'TrinidadyTobago', 'Taiwan', 'Tanzania', 'Ucrania', 'Uganda', 'Uruguay', 'Uzbekistán', 'SanVicenteylasGranadinas', 'Venezuela', 'VietNam', 'Vanuatu', 'Samoa', 'Sudáfrica', 'Zambia']

In [14]:
df_conti = pd.DataFrame({'Pais': lista_paises})

In [15]:
def asignar_continente(Pais):
    for continente, lista_paises in paises_dic.items():
        if Pais in lista_paises:
            return continente
    return None

asignar_continente(paises_dic)


In [16]:
df_conti['Continentes'] = df_conti['Pais'].apply(lambda x: asignar_continente(x))

In [17]:
df_conti

,Pais,Continentes
0,España,Europa
1,Alemania,Europa
2,ReinoUnido,Europa
3,Francia,Europa
4,Portugal,Europa
...,...,...
166,VietNam,Asia
167,Vanuatu,Oceania
168,Samoa,Asia
169,Sudáfrica,Africa


In [18]:
df_concatenado = pd.merge(df_paises, df_conti, on='Pais', how='left')

df_concatenado

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%),Continentes
0,España,2023,1260.0,€,1343.9,1260.0,8.00,Europa
1,Alemania,2023,1987.0,€,2119.3,1987.0,14.86,Europa
2,ReinoUnido,2022,1621.1,£,1952.7,1929.2,12.91,Europa
3,Francia,2023,1709.3,€,1823.1,1709.3,3.87,Europa
4,Portugal,2023,886.7,€,945.7,886.7,7.80,Europa
...,...,...,...,...,...,...,...,...
3028,Uzbekistán,2004,6530.0,л,6.7,5.4,NaN,Europa
3029,Venezuela,2004,321.0,Bs,NaN,NaN,NaN,Sudamerica
3030,VietNam,2004,290000.0,₫,18.4,NaN,NaN,Asia
3031,Sudáfrica,2004,1310.0,R,202.8,157.3,17.71,Africa


In [19]:
df_def = df_concatenado.drop_duplicates(subset=['Pais', 'Año', 'SMI Local', 'Divisa', 'Dolares($)', 'SMI(€)', 'Var SMI(%)', 'Continentes'], keep='first', inplace=False)

In [20]:
df_def

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%),Continentes
0,España,2023,1260.0,€,1343.9,1260.0,8.00,Europa
1,Alemania,2023,1987.0,€,2119.3,1987.0,14.86,Europa
2,ReinoUnido,2022,1621.1,£,1952.7,1929.2,12.91,Europa
3,Francia,2023,1709.3,€,1823.1,1709.3,3.87,Europa
4,Portugal,2023,886.7,€,945.7,886.7,7.80,Europa
...,...,...,...,...,...,...,...,...
3028,Uzbekistán,2004,6530.0,л,6.7,5.4,NaN,Europa
3029,Venezuela,2004,321.0,Bs,NaN,NaN,NaN,Sudamerica
3030,VietNam,2004,290000.0,₫,18.4,NaN,NaN,Asia
3031,Sudáfrica,2004,1310.0,R,202.8,157.3,17.71,Africa


In [21]:
df_def = df_def.replace({np.nan: None, 0: None, "": None})

In [22]:
df_def[df_def["Pais"]=="Cuba"]

,Pais,Año,SMI Local,Divisa,Dolares($),SMI(€),Var SMI(%),Continentes
43,Cuba,2020,2100.0,₱,None,None,None,Norteamerica
727,Cuba,2019,400.0,₱,None,None,None,Norteamerica
896,Cuba,2018,225.0,₱,None,None,None,Norteamerica
1065,Cuba,2017,225.0,₱,None,None,None,Norteamerica
1234,Cuba,2016,225.0,₱,None,None,None,Norteamerica
1401,Cuba,2015,225.0,₱,None,None,None,Norteamerica
1565,Cuba,2014,225.0,₱,None,None,None,Norteamerica
1726,Cuba,2013,225.0,₱,None,None,None,Norteamerica
1885,Cuba,2012,225.0,₱,None,None,None,Norteamerica
2038,Cuba,2011,225.0,₱,None,None,None,Norteamerica


## INDICE DE GINI

In [23]:
datos_gini=extraer_datos('https://datosmacro.expansion.com/demografia/indice-gini?anio=',20,2023)
datos_gini

[['España [+]', '2022', '32,0', '', '-3,03%'],
 ['Alemania [+]', '2022', '28,8', '', '-7,69%'],
 ['Reino Unido [+]', '2018', '33,5', '', '1,21%'],
 ['Francia [+]', '2021', '29,3', '', '0,34%'],
 ['Italia [+]', '2022', '32,7', '', '-0,61%'],
 ['Portugal [+]', '2021', '33,0', '', '5,77%'],
 ['Estados Unidos [+]', '2020', '39,7', '', '-4,34%'],
 ['Japón [+]', '2013', '32,9', '', '2,49%'],
 ['China [+]', '2019', '38,2', '', '-0,78%'],
 ['Emiratos Árabes Unidos [+]', '2018', '26,0', '', '-20,00%'],
 ['Albania [+]', '2020', '33,2', '', '-3,21%'],
 ['Armenia [+]', '2021', '27,9', '', '11,16%'],
 ['Angola [+]', '2018', '51,3', '', '20,14%'],
 ['Argentina [+]', '2021', '42,0', '', '-0,71%'],
 ['Austria [+]', '2022', '27,8', '', '4,12%'],
 ['Australia [+]', '2018', '34,3', '', '1,78%'],
 ['Azerbaiyán [+]', '2008', '31,8', '', '19,55%'],
 ['Bosnia y Herzegovina [+]', '2015', '32,7', '', '-0,91%'],
 ['Bangladés [+]', '2016', '32,4', '', '0,93%'],
 ['Bélgica [+]', '2022', '24,9', '', '3,32%'],
 ['B

In [24]:
columnas_gini = ["Pais", "Año", "Indice de Gini","Vacia", "Var Indice de Gini %"]
    
df_gini = crear_DataFrame(datos_gini, columnas_gini)


print(df_gini)
#Eliminaciòn de la columna "Vacia", ya que no contine datos.
df_gini = df_gini.drop("Vacia", axis = 1)
df_gini

                 Pais   Año Indice de Gini Vacia Var Indice de Gini %
0          España [+]  2022           32,0                     -3,03%
1        Alemania [+]  2022           28,8                     -7,69%
2     Reino Unido [+]  2018           33,5                      1,21%
3         Francia [+]  2021           29,3                      0,34%
4          Italia [+]  2022           32,7                     -0,61%
...               ...   ...            ...   ...                  ...
3201        Samoa [+]  2002           40,7                           
3202        Yemen [+]  1998           35,0                           
3203    Sudáfrica [+]  2000           57,8                     -4,78%
3204       Zambia [+]  2004           54,3                     29,04%
3205     Zimbabue [+]  1995           50,1                           

[3206 rows x 5 columns]


,Pais,Año,Indice de Gini,Var Indice de Gini %
0,España [+],2022,"32,0","-3,03%"
1,Alemania [+],2022,"28,8","-7,69%"
2,Reino Unido [+],2018,"33,5","1,21%"
3,Francia [+],2021,"29,3","0,34%"
4,Italia [+],2022,"32,7","-0,61%"
...,...,...,...,...
3201,Samoa [+],2002,"40,7",
3202,Yemen [+],1998,"35,0",
3203,Sudáfrica [+],2000,"57,8","-4,78%"
3204,Zambia [+],2004,"54,3","29,04%"


In [25]:
#Limpieza de la columa pais.
df_gini['Pais'] = [x.replace('[+]','').replace(' ','') for x in df_gini['Pais']]


df_gini['Pais']

0           España
1         Alemania
2       ReinoUnido
3          Francia
4           Italia
           ...    
3201         Samoa
3202         Yemen
3203     Sudáfrica
3204        Zambia
3205      Zimbabue
Name: Pais, Length: 3206, dtype: object

In [26]:
#Limpieza y conversion (a float) de la columna Var Indice de Gini (%)

#Pasamos a Nan los None y las cadenas vacias
df_gini ["Var Indice de Gini %"] = [np.nan if (x==None or x=='') else x for x in df_gini ["Var Indice de Gini %"]]

#Limpiamos los elementos de la columna, para poder luego convertirlos a Float
df_gini ["Var Indice de Gini %"] = [x.replace('%','').replace(',','.').replace('.','') if type(x)!=float else x for x in df_gini ["Var Indice de Gini %"]]
#Pasamos a Nan los None y las cadenas vacias
df_gini ["Indice de Gini"] = [np.nan if (x==None or x=='') else x for x in df_gini ["Indice de Gini"]]

#Limpiamos los elementos de la columna, para poder luego convertirlos a Float
df_gini ["Indice de Gini"] = [x.replace(',','.').replace('.','') if type(x)!=float else x for x in df_gini ["Indice de Gini"]]

     
#Finalmente pasamos a float/int el resto de valores de la tabla
df_gini ["Var Indice de Gini %"] = [float(x)/100 if type(x)!=float else x for x in df_gini ["Var Indice de Gini %"]] 
df_gini ["Indice de Gini"] = [float(x)/100 if type(x)!=float else x for x in df_gini["Indice de Gini"]] 
df_gini ["Año"] = [int(x) if type(x)!=int else x for x in df_gini ["Año"]] 


df_gini

,Pais,Año,Indice de Gini,Var Indice de Gini %
0,España,2022,3.20,-3.03
1,Alemania,2022,2.88,-7.69
2,ReinoUnido,2018,3.35,1.21
3,Francia,2021,2.93,0.34
4,Italia,2022,3.27,-0.61
...,...,...,...,...
3201,Samoa,2002,4.07,NaN
3202,Yemen,1998,3.50,NaN
3203,Sudáfrica,2000,5.78,-4.78
3204,Zambia,2004,5.43,29.04


In [27]:
lista_paises1 = df_gini ['Pais']
lista_paises1

0           España
1         Alemania
2       ReinoUnido
3          Francia
4           Italia
           ...    
3201         Samoa
3202         Yemen
3203     Sudáfrica
3204        Zambia
3205      Zimbabue
Name: Pais, Length: 3206, dtype: object

In [28]:
Paises_dic1 = {'Europa': ['España', 'Alemania', 'ReinoUnido', 'Francia', 'Italia', 'Portugal', 'Albania', 'Armenia', 'Austria', 'Azerbaiyán', 'BosniayHerzegovina', 'Bélgica', 'Bulgaria', 'Bielorrusia', 'Suiza', 'Chipre', 'Chequia', 'Dinamarca', 'Estonia', 'Finlandia', 'Georgia', 'Grecia', 'Croacia', 'Hungría', 'Irlanda', 'Israel', 'Islandia', 'Kazajistán', 'Lituania', 'Luxemburgo', 'Letonia', 'Moldavia', 'Montenegro', 'MacedoniadelNorte', 'Mongolia', 'Malta', 'PaísesBajos', 'Noruega', 'Polonia', 'Rumanía', 'Serbia', 'Rusia', 'Suecia', 'Eslovenia', 'Eslovaquia', 'Türkiye', 'Ucrania', 'Uzbekistán'],
 'Asia': ['Japón', 'China', 'EmiratosÁrabesUnidos', 'Bangladés', 'Bután', 'Indonesia', 'India', 'Irak', 'Irán', 'Jordania', 'Kirguistán', 'Camboya', 'CoreadelSur', 'Laos', 'Líbano', 'SriLanka', 'Myanmar', 'Maldivas', 'Malasia', 'Nepal', 'Filipinas', 'Pakistán', 'EstadodePalestina', 'Surinam', 'Siria', 'Tailandia', 'Tayikistán', 'TimorOriental', 'Turkmenistán', 'VietNam', 'Yemen'],
 'Norteamerica': ['EstadosUnidos', 'Belice', 'Canadá', 'RepúblicaDominicana', 'Haití', 'Jamaica', 'SantaLucía', 'México', 'TrinidadyTobago'],
 'America_Central': ['CostaRica', 'Guatemala', 'Honduras', 'Nicaragua', 'Panamá', 'ElSalvador'],
 'Sudamerica': ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Perú', 'Paraguay', 'Uruguay', 'Venezuela'],
 'Africa': ['Etiopía', 'Angola', 'BurkinaFaso', 'Burundi', 'Benin', 'Botsuana', 'RepúblicaDemocráticadelCongo', 'RepúblicaCentroafricana', 'RepúblicadelCongo', 'CostadeMarfil', 'Camerún', 'CaboVerde', 'Yibuti', 'Argelia', 'Egipto', 'Gabón', 'Ghana', 'Gambia', 'Guinea', 'Guinea-Bisáu', 'Kenia', 'Comoras', 'Liberia', 'Lesoto', 'Marruecos', 'Madagascar', 'Malí', 'Mauritania', 'Mauricio', 'Malaui', 'Mozambique', 'Namibia', 'Níger', 'Nigeria', 'Ruanda', 'Seychelles', 'Sudán', 'SierraLeona', 'Senegal', 'Somalia', 'SudándelSur', 'SantoToméyPríncipe', 'Eswatini', 'Chad', 'Togo', 'Túnez', 'Tanzania', 'Uganda', 'Sudáfrica', 'Zambia', 'Zimbabue'],
 'Oceania': ['Australia', 'Fiyi', 'EstadosFederadosdeMicronesia', 'Kiribati', 'IslasMarshall', 'Nauru', 'PapúaNuevaGuinea', 'IslasSalomón', 'Tonga', 'Tuvalu', 'Vanuatu', 'Samoa']}

In [29]:
Europa = Paises_dic1['Europa']
Asia = Paises_dic1['Asia']
Norteamerica = Paises_dic1['Norteamerica']
America_Central = Paises_dic1['America_Central']
Sudamerica = Paises_dic1['Sudamerica']
Africa = Paises_dic1['Africa']
Oceania = Paises_dic1['Oceania']

In [30]:
df_conti1 = pd.DataFrame({'Pais': lista_paises1})

In [31]:
def asignar_continente1(Pais):
    for continente, lista_paises1 in Paises_dic1.items():
        if Pais in lista_paises1:
            return continente
    return None

asignar_continente1(Paises_dic1)

In [32]:
df_conti1['Continentes'] = df_conti1['Pais'].apply(lambda x: asignar_continente1(x))
print(df_conti1)

            Pais Continentes
0         España      Europa
1       Alemania      Europa
2     ReinoUnido      Europa
3        Francia      Europa
4         Italia      Europa
...          ...         ...
3201       Samoa     Oceania
3202       Yemen        Asia
3203   Sudáfrica      Africa
3204      Zambia      Africa
3205    Zimbabue      Africa

[3206 rows x 2 columns]


In [33]:
df_concatenado1 = pd.merge(df_gini, df_conti1, on='Pais', how='left')

In [34]:
df_concatenado1

,Pais,Año,Indice de Gini,Var Indice de Gini %,Continentes
0,España,2022,3.20,-3.03,Europa
1,España,2022,3.20,-3.03,Europa
2,España,2022,3.20,-3.03,Europa
3,España,2022,3.20,-3.03,Europa
4,España,2022,3.20,-3.03,Europa
...,...,...,...,...,...
62355,Zimbabue,1995,5.01,NaN,Africa
62356,Zimbabue,1995,5.01,NaN,Africa
62357,Zimbabue,1995,5.01,NaN,Africa
62358,Zimbabue,1995,5.01,NaN,Africa


In [35]:
df_paises.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3033 entries, 0 to 3032
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pais        3033 non-null   object 
 1   Año         3033 non-null   int64  
 2   SMI Local   3024 non-null   float64
 3   Divisa      3033 non-null   object 
 4   Dolares($)  2959 non-null   float64
 5   SMI(€)      2058 non-null   float64
 6   Var SMI(%)  1959 non-null   float64
dtypes: float64(4), int64(1), object(2)
memory usage: 166.0+ KB


In [36]:
df_def1 = df_concatenado1.drop_duplicates(subset=['Pais', 'Año', 'Indice de Gini', 'Var Indice de Gini %', 'Continentes'], keep='first', inplace=False)

In [37]:
df_def1

,Pais,Año,Indice de Gini,Var Indice de Gini %,Continentes
0,España,2022,3.20,-3.03,Europa
20,Alemania,2022,2.88,-7.69,Europa
40,ReinoUnido,2018,3.35,1.21,Europa
60,Francia,2021,2.93,0.34,Europa
80,Italia,2022,3.27,-0.61,Europa
...,...,...,...,...,...
62160,Uganda,2002,4.52,5.12,Africa
62180,Uruguay,2004,4.59,2.00,Sudamerica
62220,Venezuela,2004,4.75,-1.25,Sudamerica
62280,Yemen,1998,3.50,NaN,Asia


In [59]:
df_def1 = df_def1.replace({np.nan: None, 0: None, "": None})

In [60]:
df_def1[df_def1["Pais"]=="Zimbabue"]

,Pais,Año,Indice de Gini,Var Indice de Gini %,Continentes
3186,Zimbabue,2019,5.03,13.54,Africa
19211,Zimbabue,2017,4.43,2.55,Africa
25599,Zimbabue,2011,4.32,-13.77,Africa
44516,Zimbabue,1995,5.01,None,Africa


# EPA

In [39]:
url = "https://datosmacro.expansion.com/paro-epa"
response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, "html.parser")
soup.find_all("td")

<Response [200]>


[<td><a href="https://datosmacro.expansion.com/paro-epa/espana">España [+]</a></td>,
 <td class="numero" data-value="13.3">13,3%</td>,
 <td class="hbar wdsp4"><div class="graph_hbar" style="background-color: #F1BF00; width:47.7%;" title="13,3%"></div></td>,
 <td class="numero text-neg" data-value="0.39">0,39</td>,
 <td class="numero text-pos" data-value="-0.39">-0,39</td>,
 <td class="fecha" data-value="2023-03-01">Marzo 2023</td>,
 <td><a href="https://datosmacro.expansion.com/paro-epa/alemania">Alemania [+]</a></td>,
 <td class="numero" data-value="3">3,0%</td>,
 <td class="hbar"><div class="graph_hbar" style="background-color: #FFE000; width:10.8%;" title="3,0%"></div></td>,
 <td class="numero text-pos" data-value="-0.2">-0,20</td>,
 <td class="numero text-pos" data-value="-0.3">-0,30</td>,
 <td class="fecha" data-value="2022-12-01">Diciembre 2022</td>,
 <td><a href="https://datosmacro.expansion.com/paro-epa/uk">Reino Unido [+]</a></td>,
 <td class="numero" data-value="4.9">4,9%</td

In [40]:
paises = []
tabla = []

for valor in soup.find_all("td"):
    data_value = valor.get("data-value")  
    tabla.append((valor.text, data_value))  

tamano = int((len(tabla) // 3.22))  

# Iterate over the extracted values
for i in range(0, tamano, 6):
    paises.append(tabla[i][0].replace("[+]","").replace(" ", ""))

paises

['España',
 'Alemania',
 'ReinoUnido',
 'Francia',
 'Italia',
 'Portugal',
 'EstadosUnidos',
 'Japón',
 'China',
 'Albania',
 'Armenia',
 'Angola',
 'Argentina',
 'Austria',
 'Australia',
 'Azerbaiyán',
 'BosniayHerzegovina',
 'Barbados',
 'Bangladés',
 'Bélgica',
 'Bulgaria',
 'Bolivia',
 'Brasil',
 'Bielorrusia',
 'Belice',
 'Canadá',
 'Suiza',
 'Chile',
 'Colombia',
 'CostaRica',
 'Chipre',
 'Chequia',
 'Dinamarca',
 'RepúblicaDominicana',
 'Argelia',
 'Ecuador',
 'Estonia',
 'Egipto',
 'Finlandia',
 'Georgia',
 'Grecia',
 'Guatemala',
 'Guyana',
 'HongKong',
 'Honduras',
 'Croacia',
 'Hungría',
 'Indonesia',
 'Irlanda',
 'Israel',
 'India',
 'Irán',
 'Islandia',
 'Jamaica',
 'Jordania',
 'Kirguistán',
 'CoreadelSur',
 'Kazajistán',
 'SantaLucía',
 'SriLanka',
 'Lituania',
 'Luxemburgo',
 'Letonia',
 'Marruecos',
 'Moldavia',
 'Montenegro',
 'MacedoniadelNorte',
 'Myanmar',
 'Mongolia',
 'Malta',
 'Mauricio',
 'México',
 'Malasia',
 'Nigeria',
 'Nicaragua',
 'PaísesBajos',
 'Noruega

In [41]:
lista_duraciones = []
vepas = []
epas = []
fechas=[]
tamano = int(len(tabla) // 6.4)
tamano1 = int(len(tabla) // 6.4) - 1
for x in soup.find_all("td", class_="fecha"):
    fechas.append(x.text)
for x in soup.find_all("td", class_="numero"):
    lista_duraciones.append(x.text)

for i in range(2, tamano1, 3):
    variacion = lista_duraciones[i].replace(',', '.')  
    if variacion=="":
        variacion=0
        vepas.append(float(variacion))
    else:
        vepas.append(float(variacion))

for i in range(0, tamano1, 3):
    variacion = lista_duraciones[i].replace('%', '').replace(',', '.') 
    epas.append(float(variacion))

epasM = np.array(epas)
vepasM = np.array(vepas)

Total = epasM + vepasM
Total_List=list(Total)
list(Total)
len(Total_List)

112

In [42]:
fecha_split= [] 
ano=[]
anoNumero=[]
anoposterior=[]
for string_to_split in fechas:
    splitted = string_to_split.split()  
    fecha_split.append(splitted) 
for i in range (len(fecha_split)) :
     ano.append(fecha_split[i][1])
for i in ano:
    if i=="":
        anoNumero.append(0)
    else:
        anoNumero.append(int(i))
for i in anoNumero:    
    anoposterior.append(i-1)
anoposterior       

[2022,
 2021,
 2019,
 2021,
 2021,
 2021,
 2022,
 2022,
 2013,
 2018,
 2020,
 2020,
 2021,
 2021,
 2022,
 2015,
 2021,
 2014,
 2016,
 2021,
 2021,
 2021,
 2017,
 2013,
 2020,
 2022,
 2021,
 2022,
 2021,
 2021,
 2021,
 2021,
 2021,
 2021,
 2015,
 2022,
 2021,
 2020,
 2021,
 2019,
 2021,
 2020,
 2020,
 2016,
 2010,
 2021,
 2021,
 2021,
 2021,
 2020,
 2021,
 2020,
 2021,
 2021,
 2020,
 2020,
 2021,
 2016,
 2021,
 2019,
 2021,
 2021,
 2021,
 2015,
 2021,
 2019,
 2021,
 2019,
 2020,
 2021,
 2020,
 2021,
 2016,
 2015,
 2011,
 2021,
 2021,
 2022,
 2016,
 2019,
 2020,
 2020,
 2021,
 2021,
 2016,
 2014,
 2021,
 2021,
 2020,
 2020,
 2021,
 2019,
 2021,
 2020,
 2021,
 2021,
 2014,
 2018,
 2009,
 2021,
 2016,
 2019,
 2020,
 2020,
 2016,
 2021,
 2015,
 2021,
 2013,
 2021,
 2016,
 2021,
 2022,
 2021,
 2019,
 2021,
 2021,
 2021,
 2022,
 2022,
 2018,
 2020,
 2020,
 2021,
 2021,
 2022,
 2021,
 2016,
 2021,
 2021,
 2021,
 2017,
 2020,
 2022,
 2021,
 2022,
 2021,
 2021,
 2021,
 2021,
 2021,
 2021,
 2016,

In [43]:
user_list = list(zip(paises, anoNumero, epas))
df_EPAanoNumero = pd.DataFrame(user_list,columns=['Pais','Año', 'EPA'])
#df_EPAanoNumero['Paises'].unique()
df_EPAanoNumero

,Pais,Año,EPA
0,España,2023,13.3
1,Alemania,2022,3.0
2,ReinoUnido,2020,4.9
3,Francia,2022,7.3
4,Italia,2022,7.9
...,...,...,...
107,VietNam,2022,1.4
108,Yemen,2014,13.1
109,Sudáfrica,2022,27.8
110,Zambia,2017,9.8


In [44]:
user_list = list(zip(paises, anoposterior, Total_List))
df_EPAanoNumeroPosterior = pd.DataFrame(user_list,columns=['Pais','Año', 'EPA'])
#df_EPAanoNumeroPosterior['Paises'].unique()
df_EPAanoNumeroPosterior

,Pais,Año,EPA
0,España,2022,12.91
1,Alemania,2021,2.70
2,ReinoUnido,2019,5.90
3,Francia,2021,7.00
4,Italia,2021,6.70
...,...,...,...
107,VietNam,2021,0.18
108,Yemen,2013,13.10
109,Sudáfrica,2021,25.79
110,Zambia,2016,9.80


In [45]:
dftotal = pd.concat([df_EPAanoNumero, df_EPAanoNumeroPosterior])
dftotal

,Pais,Año,EPA
0,España,2023,13.30
1,Alemania,2022,3.00
2,ReinoUnido,2020,4.90
3,Francia,2022,7.30
4,Italia,2022,7.90
...,...,...,...
107,VietNam,2021,0.18
108,Yemen,2013,13.10
109,Sudáfrica,2021,25.79
110,Zambia,2016,9.80


In [46]:
lista_paisesE = dftotal['Pais']

In [47]:
Paises_dic2 = {'Europa': ['España', 'Alemania', 'ReinoUnido', 'Francia', 'Italia', 'Portugal', 'Albania', 'Armenia', 'Austria', 'Azerbaiyán', 'BosniayHerzegovina', 'Bélgica', 'Bulgaria', 'Bielorrusia', 'Suiza', 'Chipre', 'Chequia', 'Dinamarca', 'Estonia', 'Finlandia', 'Georgia', 'Grecia', 'Croacia', 'Hungría', 'Irlanda', 'Israel', 'Islandia', 'Kirguistán', 'Kazajistán', 'Lituania', 'Luxemburgo', 'Letonia', 'Moldavia', 'Montenegro', 'MacedoniadelNorte', 'Malta', 'PaísesBajos', 'Noruega', 'Polonia', 'Rumanía', 'Serbia', 'Rusia', 'Suecia', 'Eslovenia', 'Eslovaquia', 'SanMarino', 'Türkiye', 'Ucrania'],
 'Asia': ['Japón', 'China', 'Bangladés', 'HongKong', 'Indonesia', 'India', 'Irán', 'Jordania', 'CoreadelSur', 'SriLanka', 'Myanmar', 'Mongolia', 'Malasia', 'Filipinas', 'Pakistán', 'EstadodePalestina', 'Catar', 'ArabiaSaudita', 'Singapur', 'Siria', 'Tailandia', 'Taiwan', 'VietNam', 'Yemen'],
 'Norteamerica': ['EstadosUnidos', 'Barbados', 'Belice', 'Canadá', 'RepúblicaDominicana', 'Jamaica', 'SantaLucía', 'México', 'TrinidadyTobago'],
 'America_Central': ['CostaRica', 'Guatemala', 'Honduras', 'Nicaragua', 'Panamá'],
 'Sudamerica': ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Perú', 'Paraguay', 'Uruguay', 'Venezuela'],
 'Africa': ['Angola', 'Argelia', 'Egipto', 'Marruecos', 'Mauricio', 'Nigeria', 'Ruanda', 'Seychelles', 'Senegal', 'Túnez', 'Sudáfrica', 'Zambia', 'Zimbabue'],
 'Oceania': ['Australia', 'NuevaZelanda']
}

In [48]:
Europa = Paises_dic1['Europa']
Asia = Paises_dic1['Asia']
Norteamerica = Paises_dic1['Norteamerica']
America_Central = Paises_dic1['America_Central']
Sudamerica = Paises_dic1['Sudamerica']
Africa = Paises_dic1['Africa']
Oceania = Paises_dic1['Oceania']

In [49]:
df_conti2 = pd.DataFrame({'Pais': lista_paisesE})

In [50]:
def asignar_continente2(Pais):
    for continente, lista_paisesE in Paises_dic2.items():
        if Pais in lista_paisesE:
            return continente
    return None

asignar_continente2(Paises_dic2)

In [51]:
df_conti2['Continentes'] = df_conti2['Pais'].apply(lambda x: asignar_continente2(x))
print(df_conti2)

           Pais Continentes
0        España      Europa
1      Alemania      Europa
2    ReinoUnido      Europa
3       Francia      Europa
4        Italia      Europa
..          ...         ...
107     VietNam        Asia
108       Yemen        Asia
109   Sudáfrica      Africa
110      Zambia      Africa
111    Zimbabue      Africa

[224 rows x 2 columns]


In [52]:
df_concatenado2 = pd.merge(dftotal, df_conti2, on='Pais', how='left')

In [53]:
df_concatenado2

,Pais,Año,EPA,Continentes
0,España,2023,13.30,Europa
1,España,2023,13.30,Europa
2,Alemania,2022,3.00,Europa
3,Alemania,2022,3.00,Europa
4,ReinoUnido,2020,4.90,Europa
...,...,...,...,...
443,Sudáfrica,2021,25.79,Africa
444,Zambia,2016,9.80,Africa
445,Zambia,2016,9.80,Africa
446,Zimbabue,2021,13.68,Africa


In [54]:
df_concatenado2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448 entries, 0 to 447
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Pais         448 non-null    object 
 1   Año          448 non-null    int64  
 2   EPA          448 non-null    float64
 3   Continentes  448 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 17.5+ KB


In [55]:
df_def2 = df_concatenado2.drop_duplicates(subset=['Pais', 'Año', 'EPA', 'Continentes'], keep='first', inplace=False)

In [56]:
df_def2.reset_index(drop=True, inplace=True)

In [57]:
df_def2

,Pais,Año,EPA,Continentes
0,España,2023,13.30,Europa
1,Alemania,2022,3.00,Europa
2,ReinoUnido,2020,4.90,Europa
3,Francia,2022,7.30,Europa
4,Italia,2022,7.90,Europa
...,...,...,...,...
219,VietNam,2021,0.18,Asia
220,Yemen,2013,13.10,Asia
221,Sudáfrica,2021,25.79,Africa
222,Zambia,2016,9.80,Africa


In [61]:
df_def2 = df_def2.replace({np.nan: None, 0: None, "": None})

In [62]:
df_def2[df_def2["Pais"]=="Zimbabue"]

,Pais,Año,EPA,Continentes
111,Zimbabue,2022,11.60,Africa
223,Zimbabue,2021,13.68,Africa
